In [1]:
!export HF_DATASETS_CACHE="/scratch/pt2295/cache/"

In [2]:
!export CUDA_VISIBLE_DEVICES=0,1,2,3

In [3]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/scratch/pt2295/cache/'

In [4]:
import json
import random
import re



file_path = "/scratch/pt2295/LlvmProject/automatic_prompt_engineer/data/bigbench-ii/epistemic_reasoning/task.json"

with open(file_path, 'r') as json_file:
    
    data_orig = json.load(json_file)

In [5]:
data=data_orig['examples']

In [6]:
len(data_orig['examples'])

2000

In [7]:
random.shuffle(data)

# Define the proportions for train, test, and validation sets
total_samples = len(data)
train_ratio = 0.7
test_ratio = 0.15
validation_ratio = 0.15

# Calculate the sizes of each set
train_size = int(total_samples * train_ratio)
test_size = int(total_samples * test_ratio)
validation_size = int(total_samples * validation_ratio)

# Split the data into sets
train_data = data[:train_size]
test_data = data[train_size:train_size + test_size]
validation_data = data[train_size + test_size:]

In [10]:
len(train_data)

100

In [9]:
train_data=train_data[0:100]

# Algo

In [11]:
task_description=data_orig['description']

In [12]:
task_type='entailment'

In [13]:
task_prefix=data_orig['task_prefix']
if task_type=='hyperbaton':
    task_prefix=task_prefix.strip()
    task_prefix+=" Choose only from the following options: 'a' or 'b'.\n"

In [14]:
def preprocess(text):
    return

In [15]:
def extract_answer(length,line,task_type='entailment'):
    line=line[length:]
    pattern = r'<Ans>(.*?)</Ans>|Ans:\s*([\w-]+)'
    matches = re.findall(pattern, line)
    if len(matches)==0:#right now if not match return
        return -1
    if task_type=='entailment':
        if matches =='entailment' or matches=='Entailment' or "non-entail" not in matches:
            return 1
    elif task_type=='hyperbaton':
        if matches[0][0]=='a' or matches[0][0]=='A':
            return 1
    return 0
  

In [16]:
from transformers import AutoTokenizer,AutoModelForCausalLM
import transformers
import torch

model_name = "meta-llama/Llama-2-13b-chat-hf"
custom_cache_dir = "/scratch/pt2295/cache/"
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=custom_cache_dir,use_fast=True)

# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model_name,
#     torch_dtype=torch.float16,
#     device_map="auto",
#     tokenizer=tokenizer
    
# )
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_4bit=True,cache_dir=custom_cache_dir)




/ext3/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:37<00:00, 12.61s/it]


In [53]:
def get_answer_llm(user_prompt):
# user_prompt="Determine whether one sentence entails the next. Given input: Premise: William suspects that Isabella learns that two men are in blue life jackets and sitting on innertubes with 'Crusher' written on them in the middle of a body of water. Hypothesis: Isabella learns that two men are in blue life jackets and sitting on innertubes with 'Crusher' written on them in the middle of a body of water.Identify the relation between the following premises and hypotheses, choosing from the options 'entailment' or 'non-entailment'. Put your answer within tag <Ans> and </Ans>"
    sys_prompt="You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."
    prompt=f"<s>[INST] {user_prompt} [/INST]"
    
    
   
    model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    output = model.generate(**model_inputs)

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [17]:
# model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_4bit=True)
# tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_auth_token=access_token)
# model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

# output = model.generate(**model_inputs)

# print(tokenizer.decode(output[0], skip_special_tokens=True))

In [18]:
# for seq in sequences:
#     print(f"Result: {seq['generated_text']}")
#     print()
    

In [18]:
def run_inference_step1(task_type='entailment'):
    wrong_ans_indices=[]
    not_got=[]
    for i in range(len(train_data)):
        
        prompt_t= task_description+'\n'+'Given input: '+train_data[i]['input']+'\n'+task_prefix+'Put your one-word answer choosing from the previously stated two options within tag <Ans> and </Ans>. The one word answer should be necassarily between the tags.'
        op=get_answer_llm(prompt_t)
        output_str=op#"".join([i['generated_text'] for i in op])

        ans=extract_answer(len(prompt_t),output_str,task_type)
        if ans==-1:
            not_got.append(i)
            continue
        ground_truth=1
      
        if task_type=='entailment':
            if train_data[i]['target_scores']['non-entailment']==1:
                ground_truth=0
        elif task_type=='hyperbaton':
            if train_data[i]['target_scores']['b']==1:
                ground_truth=0
        if (ans!=ground_truth):
            wrong_ans_indices.append(i)
        
    
        
    
    return wrong_ans_indices

In [19]:
wrong_answers=run_inference_step1(task_type)

/ext3/miniconda3/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


In [ ]:
# answer not only within tags, difficult to extract answer -> solved by improving prompt
#70b is good but v slow
#maybe use pipeline with datasets or use more gpus

In [20]:
def extract_hint(length,line):
    line=line[length:]
    pattern = r'<hint>(.*?)</hint>'
    matches = re.findall(pattern, line)
    if len(matches)==0:
        return ""
    return matches[0]

In [62]:
# (wrong_answers)

In [34]:
# len(not_got_append)

In [22]:
def get_hints_residual_step2():
    hints_wrong_ans={}
    for idx in wrong_answers:
        if task_type=='entailment':
            if train_data[idx]['target_scores']['entailment']==1:
                ans='entailment'
            else:
                ans='non-entailment'
        prompt_h='Given following task:'+task_description+'\n'+'Given input: '+train_data[idx]['input']+'\n'+'And its expected output: '+ans+'\n'+'List the reason or hint why its with this expected output within tag <hint> and </hint>. The hint or explaination should be necassarily between the tags.'
       
        op=get_answer_llm(prompt_h)
        hint=extract_hint(len(prompt_h),op)
       
        if hint!='':
#             hints_wrong_ans.append((idx,hint))
            hints_wrong_ans[idx]=hint
        
    return hints_wrong_ans
    

In [23]:
hints_wrong=get_hints_residual_step2()

In [24]:
hints_wrong

{0: "Note that the premise only provides information about John's perception of a specific lady, and does not provide any information about Charles's perception or what he sees.",
 1: 'Note that the word "suspects" implies doubt or uncertainty, which means that Evelyn\'s understanding of Richard\'s knowledge is not confirmed or proven. In contrast, the hypothesis states that Richard does understand something, which is a more definite and certain claim.',
 2: "The premise and hypothesis are both statements about Thomas's understanding, but they do not directly state the same thing. The premise only states that Robert suspects something about Thomas's understanding, while the hypothesis states something about what Thomas actually understands. Therefore, they do not entail each other.",
 4: ' The answer is no, there is no new information or perspective added in the hypothesis that would make it entail the premise. The two sentences are simply restating the same information, and therefore 

In [43]:
len(hints_wrong)

69

In [38]:
import random


In [39]:
def random_sampling(wrong_answers,k=20):
    selected=random.sample(wrong_answers, k)
    return selected
    

In [60]:
def get_summarise_step4(wrong_answers, hints_wrong):
    selected=random_sampling(list(hints_wrong.keys()))#HERE I HAVED USED HINTS WRONG INSTEAD OF WRONG_ANS 
    prompt_s="This is a task to "+task_description+". We have some expected input and output pairs and have asked labeler to give reason or hint for each expected output. Given following data each contains input, output and reason for the expected output, summarize a general reason for all these cases:"+'\n'
    temp_str=''
    ctr=1
    for idx in selected:
        temp=''
        if task_type=='entailment':
            if train_data[idx]['target_scores']['entailment']==1:
                ans='entailment'
            else:
                ans='non-entailment'

        temp+='Given input: '+train_data[idx]['input']+'\n'+'And its expected output: '+ans+'. And the reason for the expected output: '+hints_wrong[idx]+'\n'
        temp_str+=temp+'\n'
        
    prompt_s+=temp_str+'\n'+"Summarised reason output:"
    print(prompt_s)
    l=len(prompt_s)
    op=get_answer_llm(prompt_s)[l+8+7:]#or extract after [/INSTR]
    return op.lstrip()
        
        
        
    

In [63]:
summarised_prompt=get_summarise_step4(wrong_answers,hints_wrong)

This is a task to Determine whether one sentence entails the next. We have some expected input and output pairs and have asked labeler to give reason or hint for each expected output. Given following data each contains input, output and reason for the expected output, summarize a general reason for all these cases:
Given input: Premise: James assumes that Richard thinks that a man grabs his crotch during a political demonstration. Hypothesis: James assumes that a man grabs his crotch during a political demonstration.
And its expected output: non-entailment. And the reason for the expected output: The premise states that James assumes Richard thinks that a man grabs his crotch, but the hypothesis only states that James assumes that a man grabs his crotch. There is no assumption about Richard's thoughts or beliefs in the hypothesis, so it does not entail the premise.

Given input: Premise: Richard assumes that William believes that an elderly man is smiling while sitting in front of a ro

In [69]:
final_prompt=task_description+"\n"+'Some useful hints are: '+summarised_prompt+'\n'+'Given input: '+'<INPUT>'+'\n'+task_prefix+'Put your one-word answer choosing from the previously stated two options within tag <Ans> and </Ans>. The one word answer should be necassarily between the tags.'

In [70]:
final_prompt

"Determine whether one sentence entails the next\nSome useful hints are: Based on the given input and output pairs, the general reason for non-entailment is that the hypothesis does not provide new information or perspective that would entail the premise. The premise and hypothesis are either the same statement or do not provide sufficient information to confirm the conclusion. Therefore, the output for all the cases is non-entailment.\nGiven input: <INPUT>\nIdentify the relation between the following premises and hypotheses, choosing from the options 'entailment' or 'non-entailment'.\nPut your one-word answer choosing from the previously stated two options within tag <Ans> and </Ans>. The one word answer should be necassarily between the tags."

'Determine whether one sentence entails the next'

# Now we have to replicate all these steps for T iterations, just a for loop